In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pyperclip
import time
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

# ✅ 브라우저 창 크기 설정
WINDOW_WIDTH = 1100  # 창 너비
WINDOW_HEIGHT = 1000  # 창 높이

# ✅ 브라우저 초기화 및 로그인

def setup_browser(position_x):
    driver = webdriver.Chrome()
    driver.set_window_size(WINDOW_WIDTH, WINDOW_HEIGHT)
    driver.set_window_position(position_x, 0)  # 위치 설정
    driver.get('https://console.thebackend.io/ko/login')
    time.sleep(0.2)
    return driver

# ✅ 로그인 함수 정의
def login_to_console(driver):
    id_input = driver.find_element(By.ID, "username")
    pyperclip.copy("dksrufp0607@naver.com")
    id_input.send_keys(Keys.CONTROL, 'v')
    time.sleep(0.2)

    pw_input = driver.find_element(By.ID, "password")
    pyperclip.copy("ghkfkd159!")
    pw_input.send_keys(Keys.CONTROL, 'v')
    time.sleep(0.2)

    login_button = driver.find_element(By.CSS_SELECTOR, 'button[type="submit"]')
    login_button.click()
    time.sleep(1)
        # ✅ WebDriverWait을 사용하여 버튼이 클릭 가능할 때까지 대기
    login_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[type="submit"]'))
    )

# ✅ 콘솔 페이지 이동
def navigate_to_page(driver):
    time.sleep(1)
    article_url = 'https://console.thebackend.io/ko/project/1ea3f14d34e89530ea88b3245bc82dc17d5f52ce1554049f19fce9219a847cfce18bb8891c9ffe90bc65e2b9a3b981853fc5513c1dd200afc9590ba6bfd5fced4230647d25328849e0917641/baseGameInfo/data'
    driver.get(article_url)

# ✅ "확인" 버튼 클릭
def confirm_click(driver):
    time.sleep(1)
    confirm_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "확인")]'))
    )
    confirm_button.click()
    print("✅ 확인 버튼 클릭 완료!")

# ✅ CSV 데이터 로드 함수

def get_csv_value(row, col):
    csv_path = "C:/엑셀/Ranking_1.csv"
    
    try:
        df = pd.read_csv(csv_path)

        # ✅ 인덱스 초과 방지
        if row < df.shape[0] and col < df.shape[1]:  
            print(f"🔹 {row}행 3열 값 (출력용): {df.iloc[row, 2]}")  # 🔹 3열 값 출력 (row,2)
            return df.iloc[row, col]  # 🔹 원하는 col 값 반환
        else:
            print("⚠️ CSV 파일의 크기를 초과하는 인덱스입니다.")
            return None

    except Exception as e:
        print(f"⚠️ CSV 로드 중 오류 발생: {e}")
        return None


# ✅ 검색 실행 함수
def execute_tasks(driver, row, col):
    b2_value = get_csv_value(row, col)
    if not b2_value:
        return

    time.sleep(1)
    table_dropdown = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//div[contains(@class, "_manageInfo-data_1rntr_71")]'))
    )
    table_dropdown.click()
    print("✅ 테이블 드롭다운 클릭 완료!")

    base_data_option = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//div[@role="option"][span[text()="BASE_DATA"]]'))
    )
    base_data_option.click()
    print("✅ BASE_DATA 선택 완료!")

    time.sleep(1)
    detail_search_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//p[contains(text(), "상세 검색")]'))
    )
    detail_search_button.click()
    print("✅ 상세 검색 클릭 완료!")

    time.sleep(1)
    search_input = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.NAME, "defaultSearchValue"))
    )
    search_input.clear()
    search_input.send_keys(b2_value)
    print(f"✅ {driver} - B2 값 입력 완료!")

    time.sleep(1)
    search_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//button[contains(@class, "ui medium positive button") and @type="submit"]'))
    )
    search_button.click()
    print("✅ 검색 버튼 클릭 완료!")

    time.sleep(1)
    uuid_element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//p[contains(@class, "_link_a1mor_113")]'))
    )
    uuid_element.click()
    print("✅ UUID 클릭 완료!")

    # time.sleep(1)
    # edit_button = WebDriverWait(driver, 10).until(
    #     EC.element_to_be_clickable((By.XPATH, '//button[contains(@class, "ui mini primary button") and text()="수정"]'))
    # )
    # edit_button.click()
    # print("✅ 수정 버튼 클릭 완료!")

def e2(driver):

    elements = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div._columnBorder_1rntr_40"))
    )

    for element in elements:
        try:
            column_name = element.find_element(By.CSS_SELECTOR, "div.inline.field:nth-child(1) p").text
            data_type = element.find_element(By.CSS_SELECTOR, "div.inline.field:nth-child(2) p").text
            value = "Placeholder"

            # ✅ 1️⃣ `p` 태그에서 값 찾기
            try:
                value = element.find_element(By.CSS_SELECTOR, "div.inline.field:nth-child(3) p").text.strip()
                if value == "":
                    raise NoSuchElementException  # 값이 비어 있으면 다음 방법으로 이동
            except NoSuchElementException:
                pass

            # ✅ 2️⃣ `textarea`에서 값 찾기 (JSON 값이 있을 가능성)
            if value == "Placeholder":
                try:
                    value = element.find_element(By.TAG_NAME, "textarea").get_attribute("value").strip()
                    if value == "":
                        raise NoSuchElementException  # 값이 비어 있으면 다음 방법으로 이동
                except NoSuchElementException:
                    pass

            # ✅ 3️⃣ Ace Editor에서 값 찾기 (`#json-editor > div.ace_scroller > div`)
            if value == "Placeholder":
                try:
                    value = get_ace_editor_value(driver)
                    if value == "":
                        raise NoSuchElementException  # 값이 비어 있으면 Placeholder 유지
                except NoSuchElementException:
                    pass

            # ✅ 데이터 저장
            data.append([column_name, data_type, value])
            print([column_name, data_type, value])

        except NoSuchElementException as e:
            print(f"⚠️ 요소 찾기 실패: {e}")
            continue
def get_ace_editor_value(driver):
    try:
        # ✅ Ace Editor 내부 텍스트 가져오기 (JavaScript 실행)
        ace_value = driver.execute_script("""
            return document.querySelector("#json-editor > div.ace_scroller > div").innerText;
        """)

        return ace_value.strip() if ace_value else "Placeholder"

    except Exception as e:
        print(f"⚠️ Ace Editor 값 추출 실패: {e}")
        return "Placeholder"

def save_to_excel(data, filename):
    """
    데이터를 엑셀 파일로 저장하는 함수
    :param data: 저장할 데이터 (리스트의 리스트 형태)
    :param filename: 저장할 엑셀 파일 이름
    """
    # 데이터프레임 생성
    df = pd.DataFrame(data, columns=["Column Name", "Data Type", "Value"])
    
    # 엑셀 파일로 저장
    df.to_excel(filename, index=False, engine="openpyxl")
    print(f"데이터가 {filename}에 저장되었습니다.")

excel_filename = "3300.xlsx"#엑셀명

data = []


# ✅ 브라우저 설정 및 실행
driver1 = setup_browser(0)
#driver2 = setup_browser(WINDOW_WIDTH)

login_to_console(driver1)
#login_to_console(driver2)

navigate_to_page(driver1)
#navigate_to_page(driver2)

confirm_click(driver1)
#confirm_click(driver2)

execute_tasks(driver1, 0, 5)
#execute_tasks(driver2, 0, 5)







e2(driver1)

save_to_excel(data, excel_filename)

✅ 확인 버튼 클릭 완료!
🔹 0행 3열 값 (출력용): 슈바니
✅ 테이블 드롭다운 클릭 완료!
✅ BASE_DATA 선택 완료!
✅ 상세 검색 클릭 완료!
✅ <selenium.webdriver.chrome.webdriver.WebDriver (session="639148d0981c9cb815c07f543c438285")> - B2 값 입력 완료!
✅ 검색 버튼 클릭 완료!
✅ UUID 클릭 완료!
['event8_Receive', 'number', '39']
['purchaseCurMileage', 'object', '']
['hpGrowth2_1', 'number', '400310']
['mission_Weekly_Check', 'object', '[\n    0,\n    0,\n    0,\n    3,\n    0,\n    0,\n    0,\n    0,\n    0,\n    0,\n    0\n]']
['hero_curLevel', 'object', '[\n    0,\n    0,\n    0,\n    3,\n    0,\n    0,\n    0,\n    0,\n    0,\n    0,\n    0\n]']
['attendanceCount', 'number', '128']
['maxBoostDraw', 'number', '800']
['boxLevel', 'object', '[\n    0,\n    0,\n    0,\n    3,\n    0,\n    0,\n    0,\n    0,\n    0,\n    0,\n    0\n]']
['goods_SuperStone', 'number', '526967']
['curResearchNum', 'number', '95']
['skillAddBuff', 'number', '0']
['growth4CurCheck', 'number', '4']
['growth4TotalLevel', 'number', '1404']
['skillCoolTime', 'object', '[\n    0,\n 